<a href="https://colab.research.google.com/github/hansschaa/Machine_Learning_Exercises/blob/master/RNN/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data

In [16]:
import os

# Ruta de la carpeta
folder = 'training_data/'
seqLenght = 200
memory = 2
epochs = 500

# Lista de archivos en la carpeta
files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)) and f.endswith('.txt')]

charsList = ['$', 'B','E','>',']', 'S', 'o', '<', '[', 'b', '?', 'Q']
# Crear una tabla de traducción
translateTable = str.maketrans({char: '-' for char in charsList})

# Extraer columnas del nivel filtradas
def extractColumns(level):
    columns = []
    for i in range(len(level[0])):
        column = ''.join(fila[i] for fila in level)
        columnFiltered = column.translate(translateTable)
        columns.append(columnFiltered)
    return columns

levelColumns = []

# Itera sobre cada archivo
for file in files:
    with open(os.path.join(folder, file), 'r') as f:
        # Lee el contenido del archivo y divide el nivel por líneas
        level = f.read().strip().split('\n')
        # Llamamos a la función y guardamos las columnas en una variable
        levelColumns += extractColumns(level)

#conjunto sin duplicados
states = set(levelColumns)
states = list(states)

In [17]:
import random

# Diccionario con los estados mapeados a un índice
state_index = {state: idx for idx, state in enumerate(states)}

# Diccionario con los indices mapeados a un estado
index_states = {idx: state for state, idx in state_index.items()}

X = []

# Asignar índices a las columnas
for column in levelColumns:
    index = state_index[column]
    X.append(index)

Y = X[:]

# Desplazar la lista hacia la izquierda
Y = Y[memory:]

# Agregar un valor aleatorio al final
for _ in range(memory):
    randomValue = random.choice(Y)
    Y.append(randomValue)

print(X)
print(Y)

[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 29, 29, 29, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 47, 47, 47, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 47, 47, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 37, 37, 37, 37, 37, 37, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 37, 21, 37, 21, 21, 21, 21, 21, 22, 31, 27, 15, 21, 21, 21, 1, 1, 31, 21, 21, 21, 21, 21, 21, 21, 21, 47, 47, 47, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 31, 21, 21, 41, 35, 25, 46, 46, 46, 21, 21, 21, 21, 21, 21, 21, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 31, 27, 27, 41, 21, 21, 21, 21, 21, 21, 21, 21, 21, 1, 31, 21, 21, 21, 21, 21, 21, 21

##RNN


In [18]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Define cuántos pasos temporales se considerarán
time_steps = memory
# Define el número de características en cada paso temporal
num_features = 1


X_train = np.array(X)
Y_train = np.array(Y)

# Reformatear para que sea adecuado para una RNN
'''X_train = [    [1, 1, 1],
    [1, 1, 2],
    [1, 2, 2],
    [2, 2, 3],
    [2, 3, 4],
    [3, 4, 3],
    [4, 3, 2],
    [3, 2, 1],
    [2, 1, 1],
    [1, 1, 2],
    [1, 2, 2]
]'''
X_train = [X[i:i+time_steps] for i in range(len(X)-time_steps)]

'''Y_train = [2, 2, 3, 4, 3, 2, 1, 1, 2, 2, 3]'''
Y_train = Y[time_steps:]


# Convertir a formato tridimensional
X_train = np.array(X_train).reshape(-1, time_steps, num_features)
Y_train = np.array(Y_train)

# Verificar las dimensiones
'''print(column_indices)
print(X_train.shape)
print("----")
for i, secuencia in enumerate(X_train[:30]):
    print(f"Secuencia {i+1}: {secuencia}")
print("**")
#print(Y_train.shape)'''

X_train = np.array(X_train).reshape(-1, memory, 1)
Y_train = np.array(Y_train)

# Generar conjunto de datos para entrenar y testear
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=6)

# Arquitectura
model = Sequential([
    SimpleRNN(units=52, activation='relu', input_shape=(memory, 1)),
    Dense(units=52, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs = epochs, batch_size=12)

# Hacer predicción
predicciones = model.predict(X_test)

'''for i, secuencia in enumerate(predicciones[:30]):
    print(f"Secuencia {i+1}: {secuencia}")'''

# Suponiendo que 'predicciones' es la matriz de probabilidades que obtuviste
clases_predichas = np.argmax(predicciones, axis=1)

# Imprimir cada elemento de clases_predichas hacia el lado
for clase in clases_predichas:
    print(clase, end=' ')

Epoch 1/500
195/195 [==============================] - 1s 2ms/step - loss: 2.2614 - accuracy: 0.6533
Epoch 2/500
195/195 [==============================] - 0s 2ms/step - loss: 1.7269 - accuracy: 0.6948
Epoch 3/500
195/195 [==============================] - 0s 2ms/step - loss: 1.6634 - accuracy: 0.6961
Epoch 4/500
195/195 [==============================] - 0s 2ms/step - loss: 1.6262 - accuracy: 0.6965
Epoch 5/500
195/195 [==============================] - 0s 2ms/step - loss: 1.6018 - accuracy: 0.6948
Epoch 6/500
195/195 [==============================] - 0s 2ms/step - loss: 1.5621 - accuracy: 0.6965
Epoch 7/500
195/195 [==============================] - 0s 2ms/step - loss: 1.5284 - accuracy: 0.6978
Epoch 8/500
195/195 [==============================] - 0s 2ms/step - loss: 1.4850 - accuracy: 0.6973
Epoch 9/500
195/195 [==============================] - 0s 2ms/step - loss: 1.4568 - accuracy: 0.6961
Epoch 10/500
195/195 [==============================] - 0s 2ms/step - loss: 1.4424 - accura

In [19]:
# Convertir los índices predichos a estados usando el diccionario
sequence = [index_states[idx] for idx in clases_predichas]

def transpose_sequence(sequence):
    heights = []
    level = ""

    colCount = 0
    charCount = 0
    maxCol = len(sequence)
    maxChar = len(sequence[0])
    level = ""

    while charCount < maxChar:
        while colCount < maxCol:
            level += sequence[colCount][charCount]
            colCount += 1
        charCount += 1
        colCount = 0
        level += '\n'

    return level

resultado = transpose_sequence(sequence)
print(resultado)

print("*********************************")


# Convertir los índices predichos a estados usando el diccionario
sequence = [index_states[idx] for idx in Y_test]

resultado = transpose_sequence(sequence)
print(resultado)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------